<a href="https://colab.research.google.com/github/muajnstu/DSK-Chain-to-predict-diabeties-/blob/main/Diabeties_Prediction_with_Classical_model_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import (accuracy_score, confusion_matrix, roc_auc_score, f1_score)
from sklearn.metrics import (confusion_matrix, accuracy_score, f1_score, roc_auc_score, recall_score, precision_score)
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
    BaggingClassifier
)
from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    Perceptron,
    SGDClassifier,
    PassiveAggressiveClassifier
)
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import shap
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/muajnstu/DSK-Chain-to-predict-diabeties-/refs/heads/main/Cleaned%20data%20after%20removal%20of%20duplicate%20values.csv')
X = df.drop(columns=['Outcome'])
y = df['Outcome']

#print("Class distribution:\n", y.value_counts())
# --- Handle Imbalanced Data ---

#smote = SMOTE(random_state=42)
#X_resampled, y_resampled = smote.fit_resample(X, y)

#print("Balanced class distribution:\n", pd.Series(y_resampled).value_counts())
# --- Train/Test Split ---

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46, stratify=y)


In [3]:
# metrics function
def print_metrics(y_true, y_pred, y_prob=None):
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    num_classes = cm.shape[0]

    if num_classes == 2:
        TN, FP, FN, TP = cm.ravel()
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        gmean = np.sqrt(specificity * sensitivity)
        type1 = FP / (FP + TN) if (FP + TN) > 0 else 0
        type2 = FN / (TP + FN) if (TP + FN) > 0 else 0
        fmeasure = f1_score(y_true, y_pred, pos_label=1)
        auc = 0
        if y_prob is not None and hasattr(y_prob, "shape") and y_prob.shape[1] > 1:
            try:
                auc = roc_auc_score(y_true, y_prob[:, 1])
            except Exception:
                auc = 0
    else:
        TP = np.diag(cm)
        FP = np.sum(cm, axis=0) - TP
        FN = np.sum(cm, axis=1) - TP
        TN = np.sum(cm) - (FP + FN + TP)
        specificity = np.mean([TN[i] / (TN[i] + FP[i]) if (TN[i] + FP[i]) > 0 else 0 for i in range(num_classes)])
        sensitivity = np.mean([TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)])
        gmean = np.sqrt(specificity * sensitivity)
        type1 = np.mean([FP[i] / (FP[i] + TN[i]) if (FP[i] + TN[i]) > 0 else 0 for i in range(num_classes)])
        type2 = np.mean([FN[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)])
        fmeasure = f1_score(y_true, y_pred, average='macro')
        auc = 0
        if y_prob is not None and hasattr(y_prob, "shape") and y_prob.shape[1] > 1:
            try:
                auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro')
            except Exception:
                auc = 0

    print(f"Accuracy      : {accuracy:.4f}")
    print(f"Sensitivity   : {sensitivity:.4f}")
    print(f"Specificity   : {specificity:.4f}")
    print(f"G-Mean        : {gmean:.4f}")
    print(f"Type I Error  : {type1:.4f}")
    print(f"Type II Error : {type2:.4f}")
    print(f"F1 Score      : {fmeasure:.4f}")
    print(f"AUROC         : {auc:.4f}")

# Universal runner for any model
def run_model(name, model, X_train, X_test, y_train, y_test):
    print(f"\n===== Running {name} =====")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    try:
        y_prob = model.predict_proba(X_test)
    except AttributeError:
        y_prob = None
    print_metrics(y_test, y_pred, y_prob)

#  classical ML models
ml_models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "RidgeClassifier": RidgeClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "Perceptron": Perceptron(random_state=42),
    "SGDClassifier": SGDClassifier(random_state=42),
    "PassiveAggressive": PassiveAggressiveClassifier(random_state=42),
    #"LinearSVM": SVC(kernel='linear', probability=True, random_state=42),
    #"RBFSVM": SVC(kernel='rbf', probability=True, random_state=42),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(verbosity=-1, random_state=42)
}


# Run all models in a loop
for name, model in ml_models.items():
    run_model(name, model, X_train, X_test, y_train, y_test)


===== Running RandomForest =====
Accuracy      : 0.7666
Sensitivity   : 0.1220
Specificity   : 0.9131
G-Mean        : 0.3337
Type I Error  : 0.0869
Type II Error : 0.8780
F1 Score      : 0.1622
AUROC         : 0.5786

===== Running ExtraTrees =====
Accuracy      : 0.7605
Sensitivity   : 0.1504
Specificity   : 0.8993
G-Mean        : 0.3678
Type I Error  : 0.1007
Type II Error : 0.8496
F1 Score      : 0.1888
AUROC         : 0.5068

===== Running Bagging =====
Accuracy      : 0.7440
Sensitivity   : 0.1463
Specificity   : 0.8799
G-Mean        : 0.3588
Type I Error  : 0.1201
Type II Error : 0.8537
F1 Score      : 0.1748
AUROC         : 0.5513

===== Running GradientBoosting =====
Accuracy      : 0.8268
Sensitivity   : 0.0772
Specificity   : 0.9972
G-Mean        : 0.2775
Type I Error  : 0.0028
Type II Error : 0.9228
F1 Score      : 0.1418
AUROC         : 0.6883

===== Running AdaBoost =====
Accuracy      : 0.8148
Sensitivity   : 0.0000
Specificity   : 1.0000
G-Mean        : 0.0000
Type I Er

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [19:39:18] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy      : 0.8163
Sensitivity   : 0.1382
Specificity   : 0.9704
G-Mean        : 0.3662
Type I Error  : 0.0296
Type II Error : 0.8618
F1 Score      : 0.2179
AUROC         : 0.6875

===== Running LightGBM =====
Accuracy      : 0.8200
Sensitivity   : 0.1220
Specificity   : 0.9787
G-Mean        : 0.3455
Type I Error  : 0.0213
Type II Error : 0.8780
F1 Score      : 0.2007
AUROC         : 0.6872
